### Objective: To help test and implement algorithm that will solve monthly task problem 


At St. Francis Urgent Care center there is a need to schedule tasks on a monthly schedule and distribute tasks fairly amont the different staff members. Specificallly, if there are 20 tasks and 4 medical assistants, and each require same effort, then each should do 5 tasks. 

Important to recognize that each task:
- has a date 
- has a weight (amount of effort) 
- may require some training

Important to recognize that each staff has: 
- schedule that limits their availability
- list of availabilities 

#### CSP Modeling

The problem will be modeled as a CSP. We can choose to either make the variables the tasks, and domain of each variable the people who can complete the tasks, or the other way around. For simplicity of assignment, we will assign each people (domain) to tasks. 

Variable: Each task (note that a task may repeat, so there will one variable for each task)
Domain: The people who are able to complete the task will be fill this in. 

##### To Do:
Make dummy MA master schedule. 
Make dummy Demand schedule 


In [11]:
import numpy as np
import pandas as pd
import ast 
from __future__ import print_function
from ortools.linear_solver import pywraplp
from math import isnan

In [12]:
ma_master_schedule = pd.read_table('MA_master_schedule.tsv', delimiter='\t')
demand_schedule = pd.read_table("task_day_schedule.tsv", delimiter='\t')

In [13]:
num_cols = 0
print(demand_schedule.values.shape[0])
for row in range(demand_schedule.values.shape[0]):
    num_cols = num_cols + len(ast.literal_eval(demand_schedule.values[row, -1]))

matrix_vars = arr = np.zeros((ma_master_schedule.values.shape[0], num_cols), dtype=object)
print("matrix_vars is of shape {}".format(matrix_vars.shape))


solver = pywraplp.Solver('SolveIntegerProblem',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


14
matrix_vars is of shape (5, 56)


In [14]:
#go trhough each row col combination 
#create a variable if appropriate and store in matrix 
#MA information
ma_names = ma_master_schedule.values[:,0] 
str_ma_dates = ma_master_schedule.values[:,-1]
ma_dates_available = []
for string_dates in str_ma_dates:
    ma_dates_available = ma_dates_available + [ast.literal_eval(string_dates)]


ma_skills = ma_master_schedule.values[:,1]
for idx, skill in enumerate(ma_skills):
    if type(skill) != type(1.1):
        ma_skills[idx] = [skill]
    else:
        ma_skills[idx] = ["none"]


    
#weights, requirements 
task_weights = demand_schedule.values[:, 2]
skills_req = demand_schedule.values[:,1]
tasks = demand_schedule.values[:,0]
str_task_days = demand_schedule.values[:,-1]
task_days = []
for s in str_task_days:
    task_days = task_days + [ast.literal_eval(s)]
    
print(task_days)
task_pairs = []
task_matrix_weights =[]
for idx, task in enumerate(tasks):
    for day in task_days[idx]:
        if type(skills_req[idx]) == type(1.1):
            task_pairs = task_pairs + [(task, task_weights[idx], day, "none")]
        else:
            task_pairs = task_pairs + [(task, task_weights[idx], day, str(skills_req[idx]))]
        task_matrix_weights = task_matrix_weights + [task_weights[idx]]

for row, ma in enumerate(ma_names):  
    for col, task_pair in enumerate(task_pairs):
        if task_pair[2] in ma_dates_available[row] and task_pair[3] == "none" or task_pair[3] in ma_skills[row]:
            var_name = ma + " " + task_pair[0] + " " + str(task_pair[2])
            matrix_vars[row, col] = solver.IntVar(0.0, 1.0, var_name)
        else:
            matrix_vars[row, col] = "none"


[[30], [30], [15, 30], [30], [30], [30], [1, 8, 15, 22, 29], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [15, 30], [17], [15, 30], [15], [16], [1, 8, 15, 22, 29]]


In [15]:
#add constraints:

print("shape of matrix is {}".format(matrix_vars.shape))
z = solver.IntVar(0.0, solver.infinity(),"z")
ma_constraints = []
for row in range(matrix_vars.shape[0]):
    constraint = solver.Constraint(-solver.infinity(), 0)
    constraint.SetCoefficient(z, -1)
    for col in range(matrix_vars.shape[1]):
        if type(matrix_vars[row, col]) != type("none"):
            constraint.SetCoefficient(matrix_vars[row, col], task_matrix_weights[idx])
    ma_constraints = ma_constraints + [constraint]

shape of matrix is (5, 56)
variable name is A Replace Printing Paper 15 and weight is 0.1
variable name is A Replace Printing Paper 30 and weight is 0.1
variable name is A Send Billing paperwork 30 and weight is 0.1
variable name is A Create Monthly patient report 30 and weight is 0.1
variable name is A Runs Cures Report 30 and weight is 0.1
variable name is A Stock rooms for materials 1 and weight is 0.1
variable name is A Stock rooms for materials 15 and weight is 0.1
variable name is A Check and sort mail 1 and weight is 0.1
variable name is A Check and sort mail 2 and weight is 0.1
variable name is A Check and sort mail 3 and weight is 0.1
variable name is A Check and sort mail 5 and weight is 0.1
variable name is A Check and sort mail 7 and weight is 0.1
variable name is A Check and sort mail 9 and weight is 0.1
variable name is A Check and sort mail 10 and weight is 0.1
variable name is A Check and sort mail 14 and weight is 0.1
variable name is A Check and sort mail 15 and weigh

In [16]:
ma_constraints[0]

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x1159c8120> >

#### Converting to integer optimization problem


1. Create formula
    - objective function
    - formulas that objective function is 
2. Create the formulas that each one is subject to
3. Place them for submission



In [17]:
#be careful as you are assuming that all vlaues will be str  



ma_names = ma_master_schedule.values[:,0] 
skills = ma_master_schedule.values[:,1]
weights = demand_schedule.values[:, 2]
dates_available = ma_master_schedule.values[:,-1]
tasks = demand_schedule.values[:,0]
days = demand_schedule.values[:,-1]
task_days = []

for idx, task in enumerate(tasks):
    days_available = ast.literal_eval(demand_schedule.values[idx, -1])
    skill_needed = demand_schedule.values[idx,1]
    for day in days_available: 
        task_days = task_days + [(task, day, str(skill_needed), weights[idx])]
    
for idx, s in enumerate(skills):
    if type(s) == type(1.1):
        skills[idx] = "none"
        
for idx, date in enumerate(dates_available):
    dates_available[idx] = ast.literal_eval(dates_available[idx])
    


In [18]:
task_constraints = []
for col in range(matrix_vars.shape[1]):
    constraint = solver.Constraint(1, 1)
    for row in range(matrix_vars.shape[0]):
        if type(matrix_vars[row, col]) != type("none"):
            constraint.SetCoefficient(matrix_vars[row, col], 1)
    task_constraints = task_constraints + [constraint]

In [19]:
objective = solver.Objective()
objective.SetCoefficient(z,1)
objective.SetMinimization()
result_status = solver.Solve()

In [20]:
result_status

0

In [21]:
# for variable in all_vars:
#     print('%s = %d' % (variable.name(), variable.solution_value()))
    
for row in range(matrix_vars.shape[0]):
    for col in range(matrix_vars.shape[1]):
        if type(matrix_vars[row, col]) != type("none"):
            print('%s = %d' % (matrix_vars[row, col].name(), matrix_vars[row, col].solution_value()))

A Replace Printing Paper 15 = 0
A Replace Printing Paper 30 = 1
A Send Billing paperwork 30 = 1
A Create Monthly patient report 30 = 0
A Runs Cures Report 30 = 1
A Stock rooms for materials 1 = 0
A Stock rooms for materials 15 = 0
A Check and sort mail 1 = 0
A Check and sort mail 2 = 1
A Check and sort mail 3 = 0
A Check and sort mail 5 = 0
A Check and sort mail 7 = 0
A Check and sort mail 9 = 0
A Check and sort mail 10 = 0
A Check and sort mail 14 = 1
A Check and sort mail 15 = 1
A Check and sort mail 17 = 0
A Check and sort mail 20 = 1
A Check and sort mail 21 = 0
A Check and sort mail 23 = 0
A Check and sort mail 24 = 0
A Check and sort mail 25 = 1
A Check and sort mail 28 = 0
A Check and sort mail 30 = 0
A Check and sort mail 31 = 1
A Clean Back office 15 = 0
A Clean Back office 30 = 1
A Send paperwork to occupational health 17 = 0
A Order lab supplies  15 = 0
A Order lab supplies  30 = 0
A Order office supplies 15 = 0
A File lab paperwork 1 = 1
A File lab paperwork 15 = 0
B Replac

In [22]:
solver.Objective().Value()

2.0